In [10]:
import numpy as np
import pandas as pd
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [12]:
from data.spectrogram import import_data, import_pair_data
from data import *

In [13]:
from models.baseline import *
from models.cnn import *
from models.self_supervised import *
from models.utils import *
from models import *

In [14]:
from losses import NT_Xent
from contrastive_learning import pretrain
from train import record_log,evaluation,train,load_checkpoint,save_model

### Parameters

In [15]:
TRAINMODE = 'simclr' # option: 'simclr','normal'           
NETWORK =  'resnet'  # option: 'shallow','alexnet','vgg16','resnet'      
PAIRING =  'nuc2'    # option: 'pwr','nuc2' 
TEMPERATURE = 'L'    # option: 'L', 'M', 'H'

REGULARIZE = None    # option: True,None
JOINT = 'first'      # option: 'first','second','joint'

PRETRAIN_SKIP = ['waving'] # list of name of activities to be skiped in pretraining
FINETUNE_SKIP = ['waving'] # list of name of activities to be skiped in finetuning/training

EXTRA = None   # add extra string at the end of file for notation
VAL = 'random'   # validation method; option: 'id', 'random'

PRE_BATCH_SIZE = 64          # batch size for simclr
BATCH_SIZE = 64
NUM_WORKERS = 2
PRETRAIN_EPOCHS = 750
LAB_FINETUNE_EPOCHS = 200
Y_SAMPLING = None            # option: None,'oversampling','undersampling'
VAL_ID = 1                   # participant; option: 1,2,3,4,5


fp2 = './data/experiment_data/exp_2/spectrogram' 
fp3 = './data/experiment_data/exp_3/spectrogram' 
fp4 = './data/experiment_data/exp_4/spectrogram_multi'

OUT_PATH = './laboratory/experiment_b'
FP = fp4

In [16]:
# random seed
np.random.seed(1024)
torch.manual_seed(1024)

# gpu setting
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") # some computer set to "cuda:0"
device = DEVICE
torch.cuda.set_device(DEVICE)

In [17]:
def load_encoder(network, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    encoder.load_state_dict(torch.load(model_fp))
    return encoder, outsize

def load_model(network, freeze, lb, model_fp):
    encoder, outsize = create_encoder(network,'nuc2')
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    model.load_state_dict(torch.load(model_fp))
    return model
    

# Main

In [18]:
trainmode = TRAINMODE
pairing = PAIRING
network = NETWORK
temperature = TEMPERATURE
joint = JOINT
pretrain_skip = PRETRAIN_SKIP
activities = FINETUNE_SKIP
batch_size = BATCH_SIZE
num_workers = NUM_WORKERS
pretrain_epochs = PRETRAIN_EPOCHS 
lab_finetune_epochs = LAB_FINETUNE_EPOCHS 
regularize = REGULARIZE 
t = TEMPERATURE
y_sampling = Y_SAMPLING
val_method = VAL
validation_id = VAL_ID
pre_batch_size = PRE_BATCH_SIZE
filepath = FP

temperature_dict = {'L':0.1,'M':0.5,'H':1}    
temperature = temperature_dict[t]

if TRAINMODE == 'pwr':
    joint = 'first'

if TRAINMODE == 'normal':
    if regularize == True:
        t = 'r'
    else:
        t = ''
    pairing = 'nuc2'
    freeze=False
else:
    freeze=True

if JOINT == 'joint':
    j='-j'
else:
    j=''
    
if EXTRA == None:
    extra = ''
else:
    extra = '-Extra-'+str(EXTRA)

exp_name = f'Trainmode-{trainmode}-{t}_Network-{network}_Data-exp4csi{pairing}{j}{extra}'
model_outpath  = OUT_PATH+'/'+'saved_models'
record_outpath = OUT_PATH+'/'+'records'
print(exp_name)

Trainmode-simclr-L_Network-resnet_Data-exp4csinuc2


In [19]:
####################################################### Main #############################################################

# importing data from filepath, need to decide the pairing 
datas = import_pair_data(filepath,modal=['csi',pairing],return_id=True)

Importing Data >>>>>>> Complete


In [20]:
# filtering activities for pretraining (and finetuning)
X1,X2,y,id_ls = initial_filtering_activities(datas,activities=pretrain_skip)

In [21]:
# id for loov validation, else randomly spliting data
if val_method == 'id':
    X1_train, X1_test, X2_train, X2_test, y_train_, y_test_ = split_datasets(X1,X2,y,by='id',id_ls=id_ls,validation_id=validation_id)
else:
    X1_train, X1_test, X2_train, X2_test, y_train_, y_test_ = split_datasets(X1,X2,y,split=0.8,stratify=y)

In [22]:
################################ Lab-Pretrain-phase ################################

# create dataloader for pretraining  
pretrain_loader = create_dataloader(X1_train,X2_train,batch_size=pre_batch_size,num_workers=num_workers)
print('X1_train: ',X1_train.shape,'\tX2_train: ',X2_train.shape)

# create enocder  
encoder, outsize = create_encoder(network,'nuc2')
encoder2, outsize2 = None,None

# the pairing data use the same encoder if 'joint', else we use two encoders
if joint == 'joint':
    simclr = add_SimCLR(encoder, outsize)
else:
    encoder2, outsize2 = create_encoder(network,pairing)
    simclr = add_SimCLR_multi(enc1=encoder,enc2=encoder2,out_size1=outsize,out_size2=outsize2)
    
# (optional) pretraining with simclr  
phase = 'pretrain'
if trainmode == 'simclr':
    criterion = NT_Xent(pre_batch_size, temperature, world_size=1)
    optimizer = torch.optim.SGD(list(simclr.parameters()), lr=0.0005)
    simclr, record = pretrain(model=simclr,
                              train_loader=pretrain_loader,
                              criterion=criterion,
                              optimizer=optimizer,
                              end=pretrain_epochs,
                              device=device)
    record_log(record_outpath,exp_name,phase,record=record)
    
# save
encoder_fp = save_model(model_outpath,exp_name,phase,simclr.encoder)
del simclr,encoder,outsize, encoder2,outsize2
torch.cuda.empty_cache()

X1_train:  (477, 1, 65, 501) 	X2_train:  (477, 1, 65, 501)
Start Training
Epoch 1: >>>>>>> loss: 4.873965740203857 
Epoch 2: >>>>>>> loss: 4.776188373565674 
Epoch 3: >>>>>>> loss: 4.776790618896484 
Epoch 4: >>>>>>> loss: 4.746586799621582 
Epoch 5: >>>>>>> loss: 4.709256649017334 
Epoch 6: >>>>>>> loss: 4.6465911865234375 
Epoch 7: >>>>>>> loss: 4.63023042678833 
Epoch 8: >>>>>>> loss: 4.568632125854492 
Epoch 9: >>>>>>> loss: 4.54149055480957 
Epoch 10: >>>>>>> loss: 4.521923065185547 
Epoch 11: >>>>>>> loss: 4.435563087463379 
Epoch 12: >>>>>>> loss: 4.402491092681885 
Epoch 13: >>>>>>> loss: 4.374080181121826 
Epoch 14: >>>>>>> loss: 4.263550281524658 
Epoch 15: >>>>>>> loss: 4.312619209289551 
Epoch 16: >>>>>>> loss: 4.114223957061768 
Epoch 17: >>>>>>> loss: 4.1044135093688965 
Epoch 18: >>>>>>> loss: 4.12396240234375 
Epoch 19: >>>>>>> loss: 4.100536346435547 
Epoch 20: >>>>>>> loss: 4.057999134063721 
Epoch 21: >>>>>>> loss: 3.9668023586273193 
Epoch 22: >>>>>>> loss: 3.949614

KeyboardInterrupt: 

In [23]:
################################ Lab-Finetuning-phase ################################

# sampling condition
samplings = [1,5,10,'weight','undersampling','oversampling',]



inital = {'lab':True} # for initial record
for sampling in samplings:
    
    print('\n\nsampling: ',sampling,'\n')
    
    ### Sampling data ###
    
    # decided whether the dataset should be joined together
    X_train, X_test, y_train, y_test = select_train_test_dataset(X1_train, X1_test, X2_train, X2_test, y_train_, y_test_, joint)
    # filtering activities and label-encode the remain activities
    X_train, X_test, y_train, y_test, lb = filtering_activities_and_label_encoding(X_train, X_test, y_train, y_test, activities)
    # create dataloader class 
    lab_finetune_loader, lab_validatn_loader, class_weight = combine1(X_train, X_test, y_train, y_test, 
                                                                      sampling, lb, batch_size, num_workers, 
                                                                      y_sampling=y_sampling)
    print("class: ",lb.classes_)
    print("class_size: ",1-class_weight)
    

    # load the encoder
    encoder, outsize = load_encoder(network, encoder_fp)
    # add a classifier on top of the encoder to form the model
    model = add_classifier(encoder,in_size=outsize,out_size=len(lb.classes_),freeze=freeze)
    # (optional) simpleCNN from previous paper; model = SimpleCNN('1st',num_classes=7)
    
    
    # evaluate the untrained network  
    phase = 'lab-initial'
    if inital['lab']:
        cmtx,cls = evaluation(model,lab_finetune_loader,label_encoder=lb)
        record_log(record_outpath,exp_name,phase,cmtx=cmtx,cls=cls)
        inital['lab'] = False
    
    # finetuning 
    phase = 'lab-finetune'+'-'+str(sampling)
    criterion = nn.CrossEntropyLoss(weight=class_weight).to(device)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
    model, record = train(model=model,
                          train_loader= lab_finetune_loader,
                          criterion=criterion,
                          optimizer=optimizer,
                          end= lab_finetune_epochs,
                          test_loader = lab_validatn_loader,
                          device = device,
                          regularize = regularize)
    
    # test and record
    cmtx,cls = evaluation(model,lab_validatn_loader,label_encoder=lb)
    record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
    
    
    # (obtional) extrafinetuning: defreeze the encoder 
    if sampling == 'extrafine':
        phase = 'lab-extrafinetune'+'-'+str(sampling)
        model = unfreeze_network(model)
        optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0001)
        model, record = train(model=model,
                              train_loader= lab_finetune_loader,
                              criterion=criterion,
                              optimizer=optimizer,
                              end= lab_finetune_epochs,
                              test_loader = lab_validatn_loader,
                              device = device,
                              regularize = regularize)
        
        cmtx,cls = evaluation(model,lab_validatn_loader,label_encoder=lb)
        record_log(record_outpath,exp_name,phase,record=record,cmtx=cmtx,cls=cls,acc_rec=True)
        
    
    # every loop the model (including encoder),criterion,optimizer are to be deleted 
    if sampling != 'weight':

        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()

    elif sampling == 'weight':
        
        model_fp = save_model(model_outpath,exp_name,phase,model)
        del encoder,model,criterion,optimizer# ,record,cmtx,cls
        torch.cuda.empty_cache()
    



sampling:  1 

X_train:  (6, 1, 65, 501) 	y_train:  (6,) 	X_test:  (120, 1, 65, 501) 	y_test:  (120,)
class:  ['lay' 'pickup' 'sit' 'stand' 'standff' 'walk']
class_size:  tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667])


NameError: name 'encoder_fp' is not defined